<img src='https://theaiengineer.dev/tae_logo_gw_flat.png' alt='The Python Quants' width='35%' align='right'>


# Python & Mathematics for Data Science and Machine Learning

**© Dr. Yves J. Hilpisch | The Python Quants GmbH**<br>
AI-powered by GPT-5.x.



# Chapter 2 — Python Essentials (Math‑Centric)

This notebook mirrors the second chapter. It focuses on float behavior, shapes, views vs copies, and vectorization patterns that make math transparent and reliable in code.

Set up imports and basic configuration.


In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np  # numerical arrays and linear algebra
import matplotlib.pyplot as plt  # plotting library
from decimal import Decimal, getcontext
plt.style.use('seaborn-v0_8')
rs = np.random.default_rng(42)  # reproducible random generator
getcontext().prec = 60  # high precision for references


## Float64 epsilon and unit roundoff

We compare NumPy's epsilon with half the ULP for binary64.

In [ ]:
# Machine epsilon (gap between 1 and next float64) and unit roundoff (half)
np.finfo(np.float64).eps, 0.5 * 2.0**-52


## 0.1 + 0.2 != 0.3 (float64) vs Decimal truth

Binary floating-point cannot exactly represent 0.1 and 0.2; Decimal with sufficient precision can.

In [ ]:
0.1 + 0.2 == 0.3, Decimal('0.1') + Decimal('0.2') == Decimal('0.3')


## Views vs copies and shapes

Slicing returns a view; fancy indexing returns a copy (often changing shape).

In [ ]:
a = np.arange(6, dtype=np.float64).reshape(2, 3)
v = a[:, 1]           # view (slice)
c = a[:, [1]]         # copy (fancy index makes shape (2,1))
v[0] = 999.
a, v, c, np.shares_memory(a, v), np.shares_memory(a, c)


## Broadcasting and einsum clarity

We express the same linear map with @ and with einsum to make index roles explicit.

In [ ]:
A = np.array([[1., 2., 3.], [4., 5., 6.]])  # (2,3)
x = np.array([0.5, 1.0, -1.0])              # (3,)
y1 = A @ x
# Same result via einsum with explicit index roles: i = sum_j A_ij x_j
y2 = np.einsum('ij,j->i', A, x)
y1, y2, np.allclose(y1, y2)


## Floating-point is not associative

We show a classic counterexample where ordering changes the result due to rounding.

In [ ]:
x, y, z = 1e16, 1.0, -1e16
(x + y) + z, x + (y + z)


## Visualization: rounding error in summation (naive vs Kahan)

We compare absolute error vs a high-precision reference for naive running sum and Kahan compensated summation on many tiny positives.

Define a helper function for clarity.


In [ ]:
def kahan_sum(arr: np.ndarray) -> np.ndarray:  # function kahan_sum
    s = 0.0
    c = 0.0
    out = np.empty_like(arr, dtype=np.float64)
    for i, x in enumerate(arr):
        y = x - c
        t = s + y
        c = (t - s) - y
        s = t
        out[i] = s
    return out

n = 20000
a = rs.uniform(0, 1e-10, size=n).astype(np.float64)

# High-precision reference prefix sums
pref = []
acc = Decimal(0)
for x_ in a:
    acc += Decimal(str(x_))  # preserve decimal digits
    pref.append(acc)
ref = np.array([float(v) for v in pref], dtype=np.float64)

naive = np.cumsum(a)
kahan = kahan_sum(a)

err_naive = np.abs(naive - ref)
err_kahan = np.abs(kahan - ref)

fig, ax = plt.subplots(figsize=(6.8, 3.6), dpi=140)
x_ix = np.arange(1, n + 1)
ax.plot(x_ix, err_naive, label='naive cumsum', color='C1', lw=1.4)  # plot element
ax.plot(x_ix, err_kahan, label='Kahan sum', color='C2', lw=1.4)  # plot element
ax.set_xscale('log')  # set axis/scale
ax.set_yscale('log')  # set axis/scale
ax.set_xlabel('n (log scale)')  # set axis/scale
ax.set_ylabel('absolute error (log scale)')  # set axis/scale
ax.set_title('Rounding error in summation: naive vs Kahan')  # set axis/scale
ax.legend()  # plot element
ax.grid(alpha=0.25)  # plot element
plt.show()  # render figure


Notes

- Error growth: naive summation’s rounding error accumulates with n.
- Compensation helps: Kahan recovers low-order bits and lowers error.
- Scaling matters: smaller terms relative to the running total suffer more rounding; summing small first helps.


## Underflow/overflow and stable transforms

We show overflow in exp and fix softplus/log-sum-exp with stable formulations.

Set up imports and basic configuration.


In [ ]:
import math
import numpy as np  # numerical arrays and linear algebra
def softplus_naive(x):  # function softplus_naive
    return np.log1p(np.exp(x))
def softplus_stable(x):  # numerically stable implementation
    x = np.asarray(x, dtype=np.float64)
    pos = x > 0
    y = np.empty_like(x)
    y[pos]  = x[pos] + np.log1p(np.exp(-x[pos]))
    y[~pos] = np.log1p(np.exp(x[~pos]))
    return y
xs = np.array([-1000., -50., -1., 0., 1., 50., 1000.])
np.exp(1000), softplus_naive(xs), softplus_stable(xs)


In [ ]:
a = np.array([1000., 999., 995.])
m = a.max()
naive = np.log(np.sum(np.exp(a)))
stable = m + np.log(np.sum(np.exp(a - m)))
naive, stable


## math vs numpy and broadcasting

Compare scalar math functions to vectorized NumPy and a broadcasting distance matrix.

In [ ]:
import math
xs = [0., 1., 2., 3.]
loop_exp = [math.exp(t) for t in xs]
vec_exp = np.exp(np.array(xs))
loop_exp, vec_exp


In [ ]:
X = np.array([[0., 0.], [1., 0.], [0., 1.]])
diff = X[:, None, :] - X[None, :, :]
D = np.sqrt(np.sum(diff**2, axis=-1))
D


## Plotting for intuition: Normal histogram + PDF

Histogram approximates density; analytic PDF is the reference curve.

Create reproducible random numbers or toy data.


In [ ]:
rng = np.random.default_rng(7)  # reproducible random generator
x = rng.standard_normal(size=50_000).astype(np.float64)  # draw normal samples
fig, ax = plt.subplots(figsize=(6.4, 3.2), dpi=140)
counts, bins, _ = ax.hist(  # histogram of samples
    x, bins=80, density=True, alpha=0.35,  # bins + density
    color='C0', label='samples (hist)'
)
grid = np.linspace(bins[0], bins[-1], 600)
pdf = (1.0/np.sqrt(2*np.pi))*np.exp(-0.5*grid*grid)
ax.plot(grid, pdf, color='C1', lw=2.0, label='analytic PDF')  # plot element
ax.set_xlabel('x')  # set axis/scale
ax.set_ylabel('density')  # set axis/scale
ax.set_title('Standard Normal: histogram and analytic PDF')  # set axis/scale
ax.legend(); ax.grid(alpha=0.25)  # plot element
plt.show()  # render figure


## Plotting for intuition: Gaussian surface

A scalar field z = exp(-(x^2+y^2)) shown as a heatmap.

## Figure Generators (for reproducibility)

- `code/figures/ch02_sum_error_naive_vs_kahan.py` — rounding error plot.
- `code/figures/ch02_normal_hist_pdf.py` — normal histogram + PDF.
- `code/figures/ch02_surface_field.py` — Gaussian surface.


Plot results to visualize behavior.


In [ ]:
xs = np.linspace(-3, 3, 200)
ys = np.linspace(-3, 3, 200)
Xg, Yg = np.meshgrid(xs, ys)
Z = np.exp(-(Xg**2 + Yg**2))
fig, ax = plt.subplots(figsize=(6.4, 3.2), dpi=140)
im = ax.imshow(  # image of Z
    Z, extent=[xs.min(), xs.max(), ys.min(), ys.max()],
    origin='lower', cmap='viridis', aspect='auto'
)
fig.colorbar(im, ax=ax, shrink=0.85)
ax.set_xlabel('x'); ax.set_ylabel('y')  # set axis/scale
ax.set_title('Gaussian bump: z = exp(-(x^2 + y^2))')  # set axis/scale
plt.show()  # render figure


## Code → Math check: associativity fails in floating-point

Real addition is associative; rounded addition is not. We expose it numerically.

In [ ]:
x, y, z = 1e16, 1.0, -1e16
(x + y) + z, x + (y + z)


<img src='https://theaiengineer.dev/tae_logo_gw_flat.png' alt='The Python Quants' width='35%' align='right'>
